In [18]:
import pandas as pd

In [58]:
import numpy as np

def calc_mean_var(array):
    n = 0
    sum_xs = 0
    sum_xs2 = 0
    for num in array:
        n += 1
        sum_xs += num
        sum_xs2 += num**2
        yield {
            'n': n,
            'mean': sum_xs / n,
            'var': 0 if n == 1 else 1/(n*(n - 1)) * (n * sum_xs - sum_xs2)
        }

In [59]:
df = pd.read_csv("data/full_dataset/goemotions_1.csv")
df[['text', 'surprise']].sample(3)

,text,surprise
54340,Probably not making good pass decisions.,0
5019,You must have a different definition of exciti...,0
34737,"Here, you are among friends. I wish we all cou...",0


In [20]:
X = df['text']
y = df['surprise']

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

 

In [43]:
pipe

Pipeline(steps=[('sentenceencoder', SentenceEncoder(device=device(type='cpu'))),
                ('logisticregression', LogisticRegression())])

In [53]:
from sklearn.preprocessing import StandardScaler

pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression(),
)

weights = np.random.random(size=X.shape[0])

X, y = make_classification()
pipe.fit(X, y, 
         standardscaler__sample_weight = weights,
         logisticregression__sample_weight = weights)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [23]:
%%time 

grid.fit(X, y)

/Users/vincent/Development/mandr/venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 1.28 s, sys: 568 ms, total: 1.84 s
Wall time: 1min 4s


GridSearchCV(cv=5,
             estimator=Pipeline(memory='cache_demo',
                                steps=[('tfidfvectorizer', TfidfVectorizer()),
                                       ('truncatedsvd',
                                        TruncatedSVD(n_components=10)),
                                       ('logisticregression',
                                        LogisticRegression(C=0.1))]),
             n_jobs=2,
             param_grid={'logisticregression__C': array([  1.02329299,   3.21736253,  10.11579454,  31.80533688,
       100.        ]),
                         'truncatedsvd__n_components': [10, 20, 50, 100]},
             refit=False)

In [24]:
%%time 

grid.fit(X, y)

CPU times: user 1.21 s, sys: 513 ms, total: 1.72 s
Wall time: 41.7 s


GridSearchCV(cv=5,
             estimator=Pipeline(memory='cache_demo',
                                steps=[('tfidfvectorizer', TfidfVectorizer()),
                                       ('truncatedsvd',
                                        TruncatedSVD(n_components=10)),
                                       ('logisticregression',
                                        LogisticRegression(C=0.1))]),
             n_jobs=2,
             param_grid={'logisticregression__C': array([  1.02329299,   3.21736253,  10.11579454,  31.80533688,
       100.        ]),
                         'truncatedsvd__n_components': [10, 20, 50, 100]},
             refit=False)

Note that this tool will even allow you to do some rapid prototyping for image classification tasks!

```python
# Load embetter components
from embetter.vision import ImageLoader
from embetter.multi import ClipEncoder

# Build a pipeline as you would normally
image_emb_pipeline = make_pipeline(
    ImageLoader(convert="RGB"),
    ClipEncoder(),
    LogisticRegression()
)
# Fit the whole model as you would normally
image_emb_pipeline.fit(image_paths, image_labels)
```

I might argue though that these effects are relatively visible. But maybe some of the stuff that scikit-learn really does well is hidden. 

Time to talk about a component that really feels standard ... but really isnt!

In [41]:
# Load embetter components
from embetter.vision import ImageLoader
from embetter.multi import ClipEncoder

# Build a pipeline as you would normally
image_emb_pipeline = make_pipeline(
    ImageLoader(convert="RGB"),
    ClipEncoder(),
    LogisticRegression()
)
# Fit the whole model as you would normally
image_emb_pipeline 

Pipeline(steps=[('imageloader', ImageLoader()),
                ('clipencoder', ClipEncoder(device=device(type='cpu'))),
                ('logisticregression', LogisticRegression())])

In [62]:
inputs = np.random.normal(10e9, 1, 100)

for ex in calc_mean_var(inputs):
    print(ex)

{'n': 1, 'mean': 9999999999.50449, 'var': 0}
{'n': 2, 'mean': 10000000000.31158, 'var': -9.999999998623159e+19}
{'n': 3, 'mean': 10000000000.93202, 'var': -4.999999999432019e+19}
{'n': 4, 'mean': 10000000000.483921, 'var': -3.333333332322614e+19}
{'n': 5, 'mean': 10000000000.445856, 'var': -2.499999998972928e+19}
{'n': 6, 'mean': 10000000000.694366, 'var': -1.9999999990777463e+19}
{'n': 7, 'mean': 10000000000.677202, 'var': -1.6666666657257337e+19}
{'n': 8, 'mean': 10000000000.710833, 'var': -1.4285714276316656e+19}
{'n': 9, 'mean': 10000000000.755112, 'var': -1.2499999990637775e+19}
{'n': 10, 'mean': 10000000000.530111, 'var': -1.1111111101178024e+19}
{'n': 11, 'mean': 10000000000.331337, 'var': -9.99999998966267e+18}
{'n': 12, 'mean': 10000000000.154291, 'var': -9.090909080280526e+18}
{'n': 13, 'mean': 10000000000.251364, 'var': -8.333333322918939e+18}
{'n': 14, 'mean': 10000000000.372034, 'var': -7.69230768211082e+18}
{'n': 15, 'mean': 10000000000.39124, 'var': -7.14285713270177e+18

In [68]:
def calc_mean_alternative(array):
    n = 0
    m = None
    v = None
    for num in array:
        n += 1
        prev_m = m
        m = num if m is None else (prev_m + (num - prev_m)/n)
        v = 0 if v is None else (v + (num - prev_m)*(num - m))
        yield {
            'n': n,
            'mean': m,
            'var': 0 if n == 1 else v/(n - 1)
        }

In [70]:
inputs = np.random.normal(10e9, 1, 1000)

for ex in calc_mean_alternative(inputs):
    print(ex)

{'n': 1, 'mean': 10000000001.356434, 'var': 0}
{'n': 2, 'mean': 10000000000.902016, 'var': 0.412994369147782}
{'n': 3, 'mean': 10000000000.502647, 'var': 0.6849841728180763}
{'n': 4, 'mean': 10000000000.664988, 'var': 0.5620734308419438}
{'n': 5, 'mean': 10000000000.254498, 'var': 1.2640689447198383}
{'n': 6, 'mean': 9999999999.849174, 'var': 1.9969774014876749}
{'n': 7, 'mean': 9999999999.8477, 'var': 1.6641630511579326}
{'n': 8, 'mean': 10000000000.004484, 'var': 1.6230734757970953}
{'n': 9, 'mean': 9999999999.96823, 'var': 1.4320184522985073}
{'n': 10, 'mean': 9999999999.857498, 'var': 1.3955208945966053}
{'n': 11, 'mean': 9999999999.835405, 'var': 1.2613375589353382}
{'n': 12, 'mean': 9999999999.818506, 'var': 1.1500975340925585}
{'n': 13, 'mean': 9999999999.86137, 'var': 1.0781402419236958}
{'n': 14, 'mean': 9999999999.898054, 'var': 1.014047017671076}
{'n': 15, 'mean': 9999999999.995867, 'var': 1.085122494231525}
{'n': 16, 'mean': 10000000000.0017, 'var': 1.0133252236785968}
{'n'